<a href="https://colab.research.google.com/github/eghib22/Store-Sales-Forecasting/blob/main/model_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
! pip install kaggle
! pip install dagshub
!pip install mlflow

from google.colab import drive
drive.mount('/content/drive')

!mkdir ~/.kaggle
from google.colab import files
files.upload()
!mv "kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle/

!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip walmart-recruiting-store-sales-forecasting
!unzip '*.csv.zip'
!unzip '*.csv.zip'
!pip install mlflow dagshub lightgbm scikit-learn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85

Saving kaggle.json to kaggle.json
total 4
-rw------- 1 root root 74 Jul 16 09:25 kaggle.json
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 419MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    

Archive:  features.csv.zip
  inflating: features.csv            

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  train.csv.zip
  inflating: train.csv               

4 archives were successfully processed.
Archive:  sampleSubmission.csv.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv    

Archive:  features.csv.zip
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv            

Archive: 

In [16]:
import mlflow
import os

# Example: change this to your actual DagsHub username/repo
mlflow.set_tracking_uri("https://dagshub.com/eghib22/Store-Sales-Forecasting.mlflow")


In [17]:
with open("improved_lgbm_model.pkl", "rb") as f:
    print(f.read(20))
!ls -lh improved_lgbm_model.pkl


b'\x80\x04\x95\xe4\x01\x00\x00\x00\x00\x00\x00\x8c\x10lightgb'
-rw-r--r-- 1 root root 11M Jul 16 09:18 improved_lgbm_model.pkl


In [18]:
import joblib

model = joblib.load("improved_lgbm_model.pkl")
print("Model loaded successfully!")


Model loaded successfully!


In [19]:
def enhanced_preprocess_simple(df):
    """Simple enhanced preprocessing that works with your existing data"""
    type_map = {'A': 0, 'B': 1, 'C': 2}
    df = df.copy()

    # Handle Type column
    if 'Type' in df.columns:
        df['Type'] = df['Type'].map(type_map)

    # Handle holiday columns - check what exists
    holiday_col = None
    if 'IsHoliday_x' in df.columns:
        df['IsHoliday'] = df['IsHoliday_x'].astype(int)
        df = df.drop(columns=['IsHoliday_x'], errors='ignore')
        if 'IsHoliday_y' in df.columns:
            df = df.drop(columns=['IsHoliday_y'], errors='ignore')
        holiday_col = 'IsHoliday'
    elif 'IsHoliday' in df.columns:
        df['IsHoliday'] = df['IsHoliday'].astype(int)
        holiday_col = 'IsHoliday'
    else:
        df['IsHoliday'] = 0  # Default to no holiday
        holiday_col = 'IsHoliday'

    # Enhanced time features
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week
    df['Day'] = df['Date'].dt.day
    df['Quarter'] = df['Date'].dt.quarter
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['DayOfYear'] = df['Date'].dt.dayofyear

    # Cyclical encoding for better seasonal patterns
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['Week_sin'] = np.sin(2 * np.pi * df['Week'] / 52)
    df['Week_cos'] = np.cos(2 * np.pi * df['Week'] / 52)
    df['DayOfWeek_sin'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
    df['DayOfWeek_cos'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)

    # Handle markdowns with better features
    markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
    for col in markdown_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)
            # Create binary indicators
            df[f'{col}_Present'] = (df[col] > 0).astype(int)

    # Create markdown summary features
    if any(col in df.columns for col in markdown_cols):
        existing_markdown_cols = [col for col in markdown_cols if col in df.columns]
        df['Total_MarkDown'] = df[existing_markdown_cols].sum(axis=1)
        df['MarkDown_Count'] = (df[existing_markdown_cols] > 0).sum(axis=1)
        df['Has_MarkDown'] = (df['Total_MarkDown'] > 0).astype(int)

    # Economic indicators interactions
    if 'CPI' in df.columns and 'Unemployment' in df.columns:
        df['CPI_Unemployment_Ratio'] = df['CPI'] / (df['Unemployment'] + 0.01)
        df['CPI_Normalized'] = (df['CPI'] - df['CPI'].mean()) / df['CPI'].std()
        df['Unemployment_Normalized'] = (df['Unemployment'] - df['Unemployment'].mean()) / df['Unemployment'].std()

    # Store size features
    if 'Size' in df.columns:
        df['Size_log'] = np.log1p(df['Size'])
        df['Size_Normalized'] = (df['Size'] - df['Size'].mean()) / df['Size'].std()

    # Temperature features if available
    if 'Temperature' in df.columns:
        df['Temperature_squared'] = df['Temperature'] ** 2
        df['Temperature_Normalized'] = (df['Temperature'] - df['Temperature'].mean()) / df['Temperature'].std()

    # Fuel price features
    if 'Fuel_Price' in df.columns:
        df['Fuel_Price_log'] = np.log1p(df['Fuel_Price'])
        df['Fuel_Price_Normalized'] = (df['Fuel_Price'] - df['Fuel_Price'].mean()) / df['Fuel_Price'].std()

    # Store-Department interaction
    if 'Store' in df.columns and 'Dept' in df.columns:
        df['Store_Dept_Interaction'] = df['Store'] * 1000 + df['Dept']

    # Special dates (Christmas, Thanksgiving periods)
    df['Is_Christmas_Period'] = ((df['Month'] == 12) & (df['Day'] >= 15)).astype(int)
    df['Is_Thanksgiving_Period'] = ((df['Month'] == 11) & (df['Day'] >= 20)).astype(int)
    df['Is_Back_To_School'] = ((df['Month'] == 8) | ((df['Month'] == 9) & (df['Day'] <= 15))).astype(int)

    # Remove date column
    df = df.drop(columns=['Date'])

    # Fill any remaining NaN values
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

    return df

In [22]:

def create_lag_features(df, target_col='Weekly_Sales'):
    """Create lag features for time series"""
    if target_col not in df.columns:
        return df

    df = df.copy()
    df = df.sort_values(['Store', 'Dept', 'Date'])

    # Create lag features
    for lag in [1, 2, 4, 8]:
        df[f'Sales_Lag_{lag}'] = df.groupby(['Store', 'Dept'])[target_col].shift(lag)

    # Rolling statistics
    for window in [4, 8, 12]:
        df[f'Sales_Mean_{window}'] = df.groupby(['Store', 'Dept'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean().shift(1)
        )
        df[f'Sales_Std_{window}'] = df.groupby(['Store', 'Dept'])[target_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).std().shift(1)
        )

    # Fill NaN values created by lag features
    lag_cols = [col for col in df.columns if 'Lag_' in col or 'Mean_' in col or 'Std_' in col]
    for col in lag_cols:
        df[col] = df[col].fillna(df[col].median())

    return df

In [24]:
import numpy as np
# === Load test data ===
test_df = pd.read_csv("test.csv")  # replace with your actual file
test_df['Date'] = pd.to_datetime(test_df['Date'])
for col in ['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size', 'Type']:
    if col not in test_df.columns:
        test_df[col] = 0
# Add dummy sales column for lag (needed if missing)
if 'Weekly_Sales' not in test_df.columns:
    test_df['Weekly_Sales'] = 0

# === Preprocess ===
# Apply lag features first
test_df_with_lags = create_lag_features(test_df)
X_test = X_test.reindex(columns=model.feature_name_, fill_value=0)

# Then enhanced preprocessing
X_test = enhanced_preprocess_simple(test_df_with_lags)

# Align columns to training data (just in case):
X_test = X_test[model.feature_name_]

# Predict
preds = model.predict(X_test)
print(f"✅ Predictions complete. Sample: {preds[:5]}")


✅ Predictions complete. Sample: [2523.95604059  527.30095857 1105.50476235 1308.73121814 1269.78318212]


In [ ]:
!ls -lh lgbm_pipeline.pkl
!head -n 5 lgbm_pipeline.pkl


In [25]:
def weighted_mean_absolute_error(y_true, y_pred, is_holiday):
    """
    Computes WMAE (Weighted Mean Absolute Error)

    Parameters:
        y_true (array-like): Ground truth sales
        y_pred (array-like): Predicted sales
        is_holiday (array-like): 1 if holiday week, 0 otherwise

    Returns:
        float: WMAE score
    """
    weights = np.where(is_holiday == 1, 5, 1)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

In [26]:
wmae = weighted_mean_absolute_error(y_true, y_pred, weights)

NameError: name 'y_true' is not defined

In [27]:
import pandas as pd
import numpy as np

# Assume you already have:
# test_df: original test data (with Store, Dept, Date columns)
# preds: your predicted Weekly_Sales (1D array, same length as test_df)

# Create the "Id" column in required format
test_df['Id'] = test_df['Store'].astype(str) + "_" + \
                test_df['Dept'].astype(str) + "_" + \
                test_df['Date'].dt.strftime("%Y-%m-%d")

# Create submission DataFrame
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'Weekly_Sales': preds
})

# Ensure sorting is the same as sample submission
submission = submission.sort_values('Id')

# Save to CSV
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv created!")


✅ submission.csv created!
